In [ ]:
## import all the library will be needed in program
import time
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
from datetime import date

# f is used in Attendance function
f=1

# this function to creat new class 
def newclass(name):
    if not os.path.exists(name):
        os.makedirs(name)
# after creat new file the system can add student to  the class
    print("Do you want to add students to the class? 1-Yes 2-No")
    a=input("Enter the number of your choice: ")
    if a=="1":
        NewPerson(name)
    elif a=="2":
        print("Mission completed successfully")
    else:
        print("wrong input")


def imageFromfile(fileName):
    try:
        path = fileName  #the file that contain the images of people
        images = []  #the list that will save the images from the file 
        PeopleNames = []  #the list of names
    
        PeopleList = os.listdir(path) #the function that will take the name of the image

        #in this part, for will go throue all lists amd will append the images to list image and 
        #the name to list names
        for i in PeopleList:
            currentImg = cv2.imread(f'{path}/{i}')
            images.append(currentImg)
            PeopleNames.append(os.path.splitext(i)[0]) 
            #splitext use beacuse the name will contain the type of images
        return images,PeopleNames    
    except:
        print("This course is not available, please create a new course")

    

#this function will take the image from the image list and will convert it as matrix
#so the program will check the matrix
def findImgEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList


#this function will mark the attendance txt
def Attendance(name,courseName):
    
#this part will create the attendance file and the name will be the name of the course and date
    
    today= date.today()
    fileName= courseName+" "+str(today)+".txt"
    global f
# we use the if so the function will create only one file 
    if f==1:
        file =open(fileName,"w")
        file.write("Name,Time")
        file.close()
        f=f+1
    
# the information will be taken from the file and save it in the list
    infile = open(fileName,"r")
    nameList = []
    idList=[]
    for i in infile:
        entry = i.strip("\n").split(',')
        nameList.append(entry[0])
        idList.append(entry[1])
    infile.close()    

# will write the name of the student if it does not before    
    if name not in nameList:
        nameList.append(name)
        date1= datetime.now()
        datte= date1.strftime('%H:%M:%S')
        idList.append(datte)

    infile.close()
    out=open(fileName,"w")
    i=0
    while i< len(nameList):
        out.write(nameList[i])
        out.write(","+str(idList[i])+"\n")
        i=i+1
    out.close()



capture = cv2.VideoCapture(0)

# this function will take images = image list from imageFromFile function and PeopleNames
# course is the name of the course
def check(images,PeopleNames,course):
    #the program will ask the findImgEncodings function to find the encode of all the image list
    encodeKnownlist = findImgEncodings(images)
    
    
    while True:
        #the program take the photo and minimize the size of photo so it will not take time
        success, img = capture.read()
        imgS = cv2.resize(img,(0,0),None,0.25,0.25)
        imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
 
        facesCurrentFrame = face_recognition.face_locations(imgS)
        encodesCurrentFrame = face_recognition.face_encodings(imgS,facesCurrentFrame)
    # here the system will compare the data of the face with the data of faces in the file
        for encodeFace,faceLocation in zip(encodesCurrentFrame,facesCurrentFrame):
            matches = face_recognition.compare_faces(encodeKnownlist,encodeFace)
            faceDistance = face_recognition.face_distance(encodeKnownlist,encodeFace)
            matchIndex = np.argmin(faceDistance)
    # when the data matches Attendance function will be called
            if matches[matchIndex]:
                name = PeopleNames[matchIndex].upper()
                print(name)
                x=Attendance(name,course)
            else: 
                name = 'Unknown'
                print(name)

        cv2.imshow('Webcam',img)
        cv2.waitKey(1)
    

def NewPerson(course):
    number=int(input("How many student you want add?"))
    i=0
    #while to add different number of students
    while i<number:
#the camera will open and take photo and save it in the correct file with the name of the student
            nameOfperson=input("Please enter the name of the person: ")
            filename=course
        #the camera will take photo
            camera = cv2.VideoCapture(0)
            time.sleep(0.1)  # If you don't wait, the image will be dark
            return_value, image = camera.read()
        #the image will save in the file
            imagename= nameOfperson+"."+"png"
            fi=r'C:\Users\ajy20\Downloads' 
            path = fi + "\\" + filename
            cv2.imwrite(os.path.join(path,imagename), image)
            del(camera)
            print("new person added successfully")
            i=i+1

            
            
    message="Mission completed successfully"
    return message

def __init__(): #first function will start from program
    print("***********************")
    print("*      Welcome        *")
    print("* How can i help you? *")
    print("* 1- Add a new course   *")
    print("* 2- Add a new person   *")
    print("* 3- Take attendance  *")
    print("***********************")
    x= input("please choose the number from the list: ") #the choice of the usear
    course= input("Enter the name of the course: ")
    #if function depend on the choice of the usear and the call of every function inside 
    #the condtion
    if x=="1":
        newclass(course)
    elif x=="2":
        b=NewPerson(course)
        print(b) 
    elif x=="3":

            x,s=imageFromfile(course)
            y=check(x,s,course)
 
    #if the usear enter out of list the program eneded
    else:
        print("Wrong choice")
        print("Program ended, Thank you")
        print("***********************************************")
        
# here the first function will call        
a=__init__()

